In [2]:
"""
Created on Wed Oct 19 15:48:00 2018

@author: Daniel Cuesta
"""

import string, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

import queue as qe

from sklearn.linear_model import LinearRegression

import networkx as nx

In [3]:
def fit_plot(l, func_2_fit, size_ini, size_fin, step):
    l_func_values =[i*func_2_fit(i) for i in range(size_ini, size_fin+1, step)]
    
    lr_m = LinearRegression()
    X = np.array(l_func_values).reshape( len(l_func_values), -1 )
    lr_m.fit(X, l)
    y_pred = lr_m.predict(X)
    
    plt.plot(l, '*', y_pred, '-')

def n2_log_n(n):
    return n**2. * np.log(n)

In [4]:
l = [
[0, 10, 1, np.inf],
[np.inf, 0, 1, np.inf],
[np.inf, np.inf, 0, 1 ],
[np.inf, 1, np.inf, 0]
]

m_g = np.array(l)

def print_m_g(m_g):
    print("graph_from_matrix:\n")
    n_v = m_g.shape[0]
    for u in range(n_v):
        for v in range(n_v):
            if v != u and m_g[u, v] != np.inf:
                print("(", u, v, ")", m_g[u, v])
            
d_g = {
0: {1: 10, 2: 1}, 
1: {2: 1}, 
2: {3: 1},
3: {1: 1}
}

def print_d_g(d_g):
    print("\ngraph_from_dict:\n")
    for u in d_g.keys():
        for v in d_g[u].keys():
            print("(", u, v, ")", d_g[u][v])
            
print_m_g(m_g)
print_d_g(d_g)

graph_from_matrix:

( 0 1 ) 10.0
( 0 2 ) 1.0
( 1 2 ) 1.0
( 2 3 ) 1.0
( 3 1 ) 1.0

graph_from_dict:

( 0 1 ) 10
( 0 2 ) 1
( 1 2 ) 1
( 2 3 ) 1
( 3 1 ) 1


In [25]:
def rand_matr_pos_graph(n_nodes, sparse_factor, max_weight=50., decimals=0):
    
    """
    Genera una matriz de adyacencia a partir de un grafo dirigido ponderado.
    Devuelve la matriz de adyacencia.

    Parámetros
    ----------
    n_nodes : numero de nodos
    sparse_factor : proporcion de ramas
    max_weight : peso maximo
    """

    matriz = np.zeros((n_nodes,n_nodes))

    for i in range (0,n_nodes):
        for j in range (0,n_nodes):
            if i != j:
                aleat = np.random.rand()
                if aleat < sparse_factor:
                    aleat = np.random.randint(1, max_weight)
                    matriz[i][j] = aleat
                else:
                    matriz[i][j] = np.inf
    return matriz

def m_g_2_d_g(m_g):
    
    """
    Genera un diccionario de listas de adyacencia del grafo definido por la matriz.
    Devuelve el diccionario.

    Parámetros
    ----------
    m_g : matriz de adyacencia
    """

    dic = {}

    for i in range (0, m_g.shape[0]):
        tuplas = {}
        for j in range (0, m_g.shape[1]):
            if i != j:
                if np.isinf(m_g[i][j]) != True:
                    tuplas.update({j: m_g[i][j]})
        dic.update({i:tuplas})

    return dic
    

def d_g_2_m_g(d_g):
    
    """
    Genera una matriz de adyacencia del grafo definido por el diccionario.
    Devuelve la matriz.

    Parámetros
    ----------
    d_g : diccionario de listas de adyacencia
    """

    #recorrerDic
    keys = list(d_g.keys())
    max = keys[-1]

    matriz = np.zeros((max+1,max+1))

    #rellenamos infs
    for i in range (0,max+1):
        for j in range (0,max+1):
            if i != j:
                matriz[i][j] = np.inf

    print(d_g)
    #rellenamos los costes
    for i in d_g:
        for j in d_g[i]:
            matriz[i][j] = d_g[i][j]

    return matriz
    

In [26]:
m_g = rand_matr_pos_graph(n_nodes=5, sparse_factor=0.5, max_weight=50.)
d_g = m_g_2_d_g(m_g)
m_g_2 = d_g_2_m_g(d_g)

print_m_g(m_g)
print_d_g(d_g)
print("\nnum_elem_iguales:\t%d" % (m_g_2 == m_g).sum() )

{0: {1: 19.0, 3: 14.0}, 1: {0: 4.0, 2: 3.0, 4: 6.0}, 2: {0: 2.0}, 3: {0: 32.0, 1: 48.0, 4: 24.0}, 4: {1: 35.0}}
graph_from_matrix:

( 0 1 ) 19.0
( 0 3 ) 14.0
( 1 0 ) 4.0
( 1 2 ) 3.0
( 1 4 ) 6.0
( 2 0 ) 2.0
( 3 0 ) 32.0
( 3 1 ) 48.0
( 3 4 ) 24.0
( 4 1 ) 35.0

graph_from_dict:

( 0 1 ) 19.0
( 0 3 ) 14.0
( 1 0 ) 4.0
( 1 2 ) 3.0
( 1 4 ) 6.0
( 2 0 ) 2.0
( 3 0 ) 32.0
( 3 1 ) 48.0
( 3 4 ) 24.0
( 4 1 ) 35.0

num_elem_iguales:	25


In [91]:
def cuenta_ramas(m_g):
    """
    Cuenta el numero de ramas que hay en la matriz de adyacencia.
    Devuelve el numero de ramas.

    Parámetros
    ----------
    m_g : matriz de adyacencia
    """
    n_ramas = len(m_g[np.where(m_g != np.inf)]) - m_g.shape[0]
    
    return n_ramas

def check_sparse_factor(n_grafos, n_nodes, sparse_factor):
    """
    Genera las matrices de varios grafos aleatorios y sus proporciones.
    Devuelve la media de las proporciones.

    Parámetros
    ----------
    n_grafos : numero de grafos
    n_nodes : numero de nodos
    sparse_factor : proporcion de ramas
    """
    sparse_med = 0
    for i in range(0,n_grafos):
        m = rand_matr_pos_graph(n_nodes, sparse_factor)
        sparse_med += cuenta_ramas(m) / (n_nodes*n_nodes - n_nodes)

    return sparse_med/(i+1)

In [92]:
print(cuenta_ramas(m_g))

n_grafos=50
n_nodes=20
sparse_factor = 0.75

print("\ntrue_sparse_factor: %.3f" % sparse_factor, 
      "\nexp_sparse_factor:  %.3f" % check_sparse_factor(n_grafos=n_grafos, n_nodes=n_nodes, sparse_factor=sparse_factor))

10
49

true_sparse_factor: 0.750 
exp_sparse_factor:  0.755


In [ ]:
def save_object(obj, f_name="obj.pklz", save_path='.'):
    
    """
    Guarda un objeto Python de manera comprimida en un fichero.

    Parámetros
    ----------
    obj : objeto python
    f_name : fichero donde guardar
    save_path : direccion donde guardar
    """
    if(save_path == '.'):
        file = gzip.GzipFile(f_name, 'wb')
    else:
        file = gzip.GzipFile(save_path+f_name, 'wb')
        
    file.write(pickle.dumps(obj))
    file.close()
    
def read_object(f_name, save_path='.'):
    
    """
    Devuelve un objeto Python guardado en un fichero.

    Parámetros
    ----------
    f_name : fichero donde leer
    save_path : direccion donde guardar
    """
    if(save_path == '.'):
        file = gzip.GzipFile(f_name, 'rb')
    else:
        file = gzip.GzipFile(save_path+f_name, 'rb')
    
    buffer = ""
    while True:
        data = file.read()
        if data == "":
            break
        buffer += data
        
    obj = pickle.loads(buffer)
    file.close()
    
    return obj